In [11]:
import requests
import os
import pandas as pd
import numpy as np
import time
import sys

In [12]:
CARTO_URL = 'https://{}.carto.com/api/v2/sql'
CARTO_USER = 'wri-01'
CARTO_KEY = os.environ.get('CARTO_KEY')
def sendSql(sql, user=None, key=None):
    '''Send arbitrary sql and return response object or False'''
    user = user or CARTO_USER
    key = key or CARTO_KEY
    url = CARTO_URL.format(user)
    payload = {
        'api_key': key,
        'q': sql,
    }
    r = requests.post(url, json=payload)
    if (r.status_code >= 400):
        try:
            msg = r.json()['error'][0]
        except:
            r.raise_for_status()
        raise Exception(msg)
    return r.json()

In [13]:
# get datasets
gadm_table = 'gadm36_adm2'
wdpa_table = 'wdpa_protected_areas'

# look at fields
sendSql("select * from {} limit 0".format(gadm_table))

{'rows': [],
 'time': 0.001,
 'fields': {'cartodb_id': {'type': 'number'},
  'the_geom': {'type': 'geometry'},
  'the_geom_webmercator': {'type': 'geometry'},
  'iso': {'type': 'string'},
  'name_0': {'type': 'string'},
  'gid_1': {'type': 'string'},
  'name_1': {'type': 'string'},
  'nl_name_1': {'type': 'string'},
  'gid_2': {'type': 'string'},
  'name_2': {'type': 'string'},
  'varname_2': {'type': 'string'},
  'nl_name_2': {'type': 'string'},
  'type_2': {'type': 'string'},
  'engtype_2': {'type': 'string'},
  'cc_2': {'type': 'string'},
  'hasc_2': {'type': 'string'},
  'bbox': {'type': 'string'},
  'centroid': {'type': 'string'},
  'area': {'type': 'number'},
  'area_ha': {'type': 'number'},
  'gid_0': {'type': 'string'}},
 'total_rows': 0}

In [14]:
sendSql("select * from {} limit 0".format(wdpa_table))

{'rows': [],
 'time': 0.002,
 'fields': {'cartodb_id': {'type': 'number'},
  'the_geom': {'type': 'geometry'},
  'the_geom_webmercator': {'type': 'geometry'},
  'wdpaid': {'type': 'number'},
  'pa_def': {'type': 'string'},
  'name': {'type': 'string'},
  'orig_name': {'type': 'string'},
  'desig': {'type': 'string'},
  'desig_eng': {'type': 'string'},
  'desig_type': {'type': 'string'},
  'iucn_cat': {'type': 'string'},
  'int_crit': {'type': 'string'},
  'marine': {'type': 'string'},
  'rep_m_area': {'type': 'number'},
  'gis_m_area': {'type': 'number'},
  'rep_area': {'type': 'number'},
  'gis_area': {'type': 'number'},
  'no_take': {'type': 'string'},
  'no_tk_area': {'type': 'number'},
  'status': {'type': 'string'},
  'status_yr': {'type': 'number'},
  'gov_type': {'type': 'string'},
  'own_type': {'type': 'string'},
  'mang_auth': {'type': 'string'},
  'mang_plan': {'type': 'string'},
  'verif': {'type': 'string'},
  'metadataid': {'type': 'number'},
  'sub_loc': {'type': 'string

In [15]:
# test area intersection calculation 
# Since the two tables are in the same DB, its probably faster to join them than to 
# dl the geometry and insert it into the query. Tho we would do the latter for a more robust pipeline
#
# notes: 
#  geometry should be cast to geography for equal-area calculation
#  use fast sphere ST_AREA(geog, false)

i = 7
query = """
SELECT ST_AREA(ST_UNION(
    ARRAY(
        SELECT
            ST_INTERSECTION(a.the_geom, b.the_geom) AS the_geom
        FROM {} AS a, (
            SELECT the_geom
            FROM {} WHERE cartodb_id = {}
        ) AS b
        WHERE a.the_geom && b.the_geom
    )
)::geography, false) as area
""".format(wdpa_table, gadm_table, i)
sendSql(query)['rows'][0]['area']

In [16]:
query = """
SELECT iso, gid_2, cartodb_id, st_area(the_geom::geography, false) as adm2_area FROM {}
""".format(gadm_table)

ADM1 = pd.DataFrame(sendSql(query)['rows'])
ADM1.head()

,adm2_area,cartodb_id,gid_2,iso
0,1.724596e+08,23852,MEX.31.93_1,MEX
1,3.319101e+09,1887,AUT.6.8_1,AUT
2,1.218112e+08,23854,MEX.31.95_1,MEX
3,1.120528e+09,34974,RUS.59.7_1,RUS
4,4.633483e+09,34827,RUS.55.31_1,RUS


In [17]:
ADM1['wdpa_area'] = np.nan

In [18]:
# iterate through adm1 polygons and calculate area intersection
# some queries are likely to fail; track these so we can try again on just the failed ones.

for i in ADM1[ADM1['wdpa_area'].isna()].index:
    cdbid = ADM1.iloc[i]['cartodb_id']
    query = """
    SELECT ST_AREA(ST_UNION(
        ARRAY(
            SELECT
                ST_INTERSECTION(a.the_geom, b.the_geom) AS the_geom
            FROM {} AS a, (
                SELECT ST_MAKEVALID(the_geom) AS the_geom
                FROM {} WHERE cartodb_id = {}
            ) AS b
            WHERE a.the_geom && b.the_geom
        )
    )::geography, false) as area
    """.format(wdpa_table, gadm_table, cdbid)

    try:
        # No intersection results in null; count as 0
        area = sendSql(query)['rows'][0]['area'] or 0
        ADM1.at[i, 'wdpa_area'] = area
        # just showing progress...
        sys.stdout.write("{}/{}: {} m3 \t\t\t\r".format(i, len(ADM1), area))
        sys.stdout.flush()
    except Exception as e:
        print('{}/{} Query failed: {}'.format(i, len(ADM1), e))

print('\nNum failed: {}'.format(ADM1['wdpa_area'].isna().sum()))
ADM1.to_csv('Data/adm2_wdpa_intersection.csv')
ADM1.head()

150/2038 Query failed: Error performing intersection: InterruptedException: Interrupted!
189/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
283/2038 Query failed: Error performing intersection: InterruptedException: Interrupted!
285/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
287/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
479/2038 Query failed: Error performing intersection: InterruptedException: Interrupted!
500/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
503/2038 Query failed: You are over platform's limits: SQL query timeout erro

,adm2_area,cartodb_id,gid_2,iso,wdpa_area
0,1.724596e+08,23852,MEX.31.93_1,MEX,3.917377e+07
1,3.319101e+09,1887,AUT.6.8_1,AUT,2.541045e+09
2,1.218112e+08,23854,MEX.31.95_1,MEX,2.105262e+06
3,1.120528e+09,34974,RUS.59.7_1,RUS,0.000000e+00
4,4.633483e+09,34827,RUS.55.31_1,RUS,2.983450e+08


In [23]:
ADM1[ADM1['wdpa_area'].isna()]

,adm2_area,cartodb_id,gid_2,iso,wdpa_area
150,7.934537e+09,27142,NZL.19.1_1,NZL,NaN
189,2.069654e+10,37082,SWE.10.9_1,SWE,NaN
283,3.357786e+10,41015,USA.2.21_1,USA,NaN
285,9.653660e+10,41017,USA.2.23_1,USA,NaN
287,5.343368e+09,41255,USA.10.11_1,USA,NaN
479,9.018167e+08,11759,DNK.4.3_1,DNK,NaN
500,2.175400e+10,14108,ESP.11.1_1,ESP,NaN
503,1.984636e+10,14109,ESP.11.2_1,ESP,NaN
531,6.642145e+09,14649,GBR.1.26_1,GBR,NaN
570,1.754631e+09,14243,EST.9.1_1,EST,NaN


In [25]:
# if the bottleneck is intersection, try simplifing the query polygon?

tolerance = .0008333 # wgs84 -> degrees (100m)
for i in ADM1[ADM1['wdpa_area'].isna()].index:
    cdbid = ADM1.iloc[i]['cartodb_id']
    query = """
    SELECT ST_AREA(ST_UNION(
        ARRAY(
            SELECT
                ST_INTERSECTION(
                    ST_MAKEVALID(ST_SIMPLIFY(a.the_geom, {tolerance})),
                    b.the_geom) AS the_geom
            FROM {} AS a, (
                SELECT ST_MAKEVALID(ST_SIMPLIFY(the_geom, {tolerance})) AS the_geom
                FROM {} WHERE cartodb_id = {}
            ) AS b
            WHERE a.the_geom && b.the_geom
        )
    )::geography, false) as area
    """.format(wdpa_table, gadm_table, cdbid, tolerance=tolerance)
    
    try:
        # No intersection results in null; count as 0
        area = sendSql(query)['rows'][0]['area'] or 0
        ADM1.at[i, 'wdpa_area'] = area
        # just showing progress...
        sys.stdout.write("{}/{}: {} m3 \t\t\t\r".format(i, len(ADM1), area))
        sys.stdout.flush()
    except Exception as e:
        print('{}/{} Query failed: {}'.format(i, len(ADM1), e))

print('\nNum failed: {}'.format(ADM1['wdpa_area'].isna().sum()))
ADM1.to_csv('Data/adm2_wdpa_intersection.csv')
ADM1[ADM1['wdpa_area'].isna()]

189/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
925/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
1341/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
1451/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
1452/2038 Query failed: Error performing intersection: InterruptedException: Interrupted!
1917/2038 Query failed: You are over platform's limits: SQL query timeout error. Refactor your query before running again or contact CARTO support for more details.
1966/2038 Query failed: You are over platform's limits: SQL query timeout error. Refact

,adm2_area,cartodb_id,gid_2,iso,wdpa_area
0,1.724596e+08,23852,MEX.31.93_1,MEX,3.917377e+07
1,3.319101e+09,1887,AUT.6.8_1,AUT,2.541045e+09
2,1.218112e+08,23854,MEX.31.95_1,MEX,2.105262e+06
3,1.120528e+09,34974,RUS.59.7_1,RUS,0.000000e+00
4,4.633483e+09,34827,RUS.55.31_1,RUS,2.983450e+08


In [26]:
import psycopg2
# setup postgis connecton funcitons
pg_user=os.environ.get('POSTGIS_ENV_POSTGRES_USER')
pg_pass=os.environ.get('POSTGIS_ENV_POSTGRES_PASSWORD')
pg_host=os.environ.get('POSTGIS_PORT_5432_TCP_ADDR')
pg_port=os.environ.get('POSTGIS_PORT_5432_TCP_PORT')

def postgis(query):
    conn = psycopg2.connect(user=pg_user, 
                        password=pg_pass,
                        host=pg_host,
                        port=pg_port
                       )
    cur = conn.cursor()
    cur.execute(query)
    r = list(cur.fetchall())
    cur.close()
    conn.close()
    return r

In [27]:
# if the bottleneck is union (dissolve), try simplifying and downloading the intersected polygons
# union offline

for i in ADM1[ADM1['wdpa_area'].isna()].index:
    cdbid = ADM1.iloc[i]['cartodb_id']
    
    #first get the intersected 
    query = """
        SELECT
            ST_INTERSECTION(
                ST_MAKEVALID(ST_SIMPLIFY(a.the_geom, {tolerance})),
                b.the_geom) AS the_geom
        FROM {} AS a, (
            SELECT ST_MAKEVALID(ST_SIMPLIFY(the_geom, {tolerance})) AS the_geom
            FROM {} WHERE cartodb_id = {}
        ) AS b
        WHERE a.the_geom && b.the_geom
    """.format(wdpa_table, gadm_table, cdbid, tolerance=tolerance)
    try:
        rows = sendSql(query)['rows']
        geoms = ','.join(["'{}'::geometry".format(r['the_geom']) for r in rows if r['the_geom']])
        if len(geoms):
            query = "SELECT ST_AREA(ST_UNION(ARRAY[{}])::geography, false)".format(geoms)
            area = postgis(query)[0][0]
        else:
            area = 0
        ADM1.at[i, 'wdpa_area'] = area

        # just showing progress...
        sys.stdout.write("{}/{}: {} m3 \t\t\t\r".format(i, len(ADM1), area))
        sys.stdout.flush()

    except Exception as e:
        print('{}/{} Query failed: {}'.format(i, len(ADM1), e))
    
print('\nNum failed: {}'.format(ADM1['wdpa_area'].isna().sum()))
ADM1['perc_wdpa'] = ADM1['wdpa_area']/ADM1['adm2_area']
ADM1.to_csv('Data/adm2_wdpa_intersection.csv')
ADM1.head()

1452/2038 Query failed: Error performing intersection: InterruptedException: Interrupted!
2037/2038: 974809.46297931 m3 				
Num failed: 1


,adm2_area,cartodb_id,gid_2,iso,wdpa_area,perc_wdpa
0,1.724596e+08,23852,MEX.31.93_1,MEX,3.917377e+07,0.227148
1,3.319101e+09,1887,AUT.6.8_1,AUT,2.541045e+09,0.765582
2,1.218112e+08,23854,MEX.31.95_1,MEX,2.105262e+06,0.017283
3,1.120528e+09,34974,RUS.59.7_1,RUS,0.000000e+00,0.000000
4,4.633483e+09,34827,RUS.55.31_1,RUS,2.983450e+08,0.064389


In [37]:
cdbid = ADM1.iloc[1452]['cartodb_id']
query = """
        SELECT
            ST_INTERSECTION(
                ST_MAKEVALID(ST_SIMPLIFY(a.the_geom, {tolerance})),
                b.the_geom) AS the_geom
        FROM {} AS a, (
            SELECT ST_MAKEVALID(ST_SIMPLIFY(the_geom, {tolerance})) AS the_geom
            FROM {} WHERE cartodb_id = {}
        ) AS b
        WHERE a.the_geom && b.the_geom
    """.format(wdpa_table, gadm_table, cdbid, tolerance=tolerance*10)
rows = sendSql(query)['rows']
geoms = ','.join(["'{}'::geometry".format(r['the_geom']) for r in rows if r['the_geom']])
if len(geoms):
    query = "SELECT ST_AREA(ST_UNION(ARRAY[{}])::geography, false)".format(geoms)
    area = postgis(query)[0][0]
else:
    area = 0
area

38591101046.0429

In [39]:
ADM1.iloc[1452]

adm2_area     5.67532e+10
cartodb_id           9126
gid_2          CHL.11.4_1
iso                   CHL
wdpa_area             NaN
perc_wdpa             NaN
Name: 1452, dtype: object